In [3]:
# reading the dataset 
import csv
with open('iris.data') as csvfile:
    lines = csv.reader(csvfile)
    for row in lines:
        print(', '.join(row))

5.1, 3.5, 1.4, 0.2, Iris-setosa
4.9, 3.0, 1.4, 0.2, Iris-setosa
4.7, 3.2, 1.3, 0.2, Iris-setosa
4.6, 3.1, 1.5, 0.2, Iris-setosa
5.0, 3.6, 1.4, 0.2, Iris-setosa
5.4, 3.9, 1.7, 0.4, Iris-setosa
4.6, 3.4, 1.4, 0.3, Iris-setosa
5.0, 3.4, 1.5, 0.2, Iris-setosa
4.4, 2.9, 1.4, 0.2, Iris-setosa
4.9, 3.1, 1.5, 0.1, Iris-setosa
5.4, 3.7, 1.5, 0.2, Iris-setosa
4.8, 3.4, 1.6, 0.2, Iris-setosa
4.8, 3.0, 1.4, 0.1, Iris-setosa
4.3, 3.0, 1.1, 0.1, Iris-setosa
5.8, 4.0, 1.2, 0.2, Iris-setosa
5.7, 4.4, 1.5, 0.4, Iris-setosa
5.4, 3.9, 1.3, 0.4, Iris-setosa
5.1, 3.5, 1.4, 0.3, Iris-setosa
5.7, 3.8, 1.7, 0.3, Iris-setosa
5.1, 3.8, 1.5, 0.3, Iris-setosa
5.4, 3.4, 1.7, 0.2, Iris-setosa
5.1, 3.7, 1.5, 0.4, Iris-setosa
4.6, 3.6, 1.0, 0.2, Iris-setosa
5.1, 3.3, 1.7, 0.5, Iris-setosa
4.8, 3.4, 1.9, 0.2, Iris-setosa
5.0, 3.0, 1.6, 0.2, Iris-setosa
5.0, 3.4, 1.6, 0.4, Iris-setosa
5.2, 3.5, 1.5, 0.2, Iris-setosa
5.2, 3.4, 1.4, 0.2, Iris-setosa
4.7, 3.2, 1.6, 0.2, Iris-setosa
4.8, 3.1, 1.6, 0.2, Iris-setosa
5.4, 3.4

In [69]:
# spliting the dataset for testing and taining sets
import csv
import random
def loaddataset(filename, split, trainingSet=[], testSet=[]):
    with open(filename, 'r') as csvfile:
        lines = csv.reader(csvfile)
        dataset = list(lines)
        print("DataSet Size: "+ repr(len(dataset)))
        for x in range(len(dataset)-1):
            for y in range(4):
                dataset[x][y]=float(dataset[x][y])
            if random.random() < split:
                trainingSet.append(dataset[x])
            else:
                testSet.append(dataset[x])

In [70]:
trainingset=[]
testingset=[]
loaddataset(r'iris.data', 0.66, trainingset, testingset)
print('Train: ' + repr(len(trainingset)))
print('Test: ' + repr(len(testingset)))

DataSet Size: 151
Train: 96
Test: 54


In [71]:
# moving towards step 2, similarity.
# since all the 4 value in the dataset are numeric so there is no problem in using euclidian distance measure
# defining the function to calculate euclidian distance for multiple instances dataset
import math
def euclidianDist(instance1, instance2, length):
    distance=0
    for x in range(length):
        distance+= pow(instance1[x]-instance2[x], 2)
    return math.sqrt(distance)

In [72]:
data1 = [2,2,2,'a'] # first instance
data2 = [4,4,4,'b'] # second instance
distance = euclidianDist(data1, data2, 3)
print('Distance: ' + repr(distance))

Distance: 3.4641016151377544


In [73]:
# now we have to look for k nearest neighbors.
# selecting the smallest nearest value
# defining the function which will return the k most similar nabours from the taining set for a given test instance
import operator
def getNeighbours(trainingSet, testInstance, k):
    distance = []
    length = len(testInstance)-1
    for x in range(len(trainingSet)):
        dist = euclidianDist(testInstance, trainingSet[x], length)
        distance.append((trainingSet[x], dist))
    distance.sort(key=operator.itemgetter(1))
    neighbors=[]
    for x in range(k):
        neighbors.append(distance[x][0])
    return(neighbors)

In [74]:
trainigSet = [[2,2,2,'a'],[4,4,4,'b']]
testInstance = [5,5,5]
k=1
neighbors = getNeighbours(trainigSet, testInstance, 1)
print(neighbors)

[[4, 4, 4, 'b']]


In [75]:
# next task is to predict the response based on those neighbors, by allowing each neighbors to vote for there class attribute and take the majority vote as our response
import operator
def getResponse(neighbors):
    classvotes = {}
    for x in range(len(neighbors)):
        response = neighbors[x][-1]
        if response in classvotes:
            classvotes[response]+=1
        else:
            classvotes[response]=1
    sortedvotes = sorted(classvotes.items(), key = operator.itemgetter(1), reverse=True)
    return sortedvotes[0][0]
 

In [76]:
neighbors = [[1,1,1,'a'],[2,2,2,'a'],[3,3,3,'b']]
response = getResponse(neighbors)
print(response)

a


In [104]:
# evaluating the accuracy of the prediction
def getAccuracy(testSet, predictions):
    correct = 0
    for x in range(len(testSet)):
        if testSet[x][-1] == predictions[x]:
            correct+=1
    return (correct/float(len(testSet)))*100.0

In [105]:
testSet = [[1,1,1,'a'],[2,2,2,'a'],[3,3,3,'b']]
predictions=['a','a','a']
accuracy = getAccuracy(testSet, predictions)
print(accuracy)

66.66666666666666


In [106]:
#putting everything together in main function

def main():
    # preparing data
    trainingSet=[]
    testSet=[]
    split=0.67
    loaddataset('iris.data', split, trainingSet, testSet)
    print('Train set: '+ repr(len(trainingSet)))
    print('Test set: '+ repr(len(testSet)))
    # generate predictions
    predictions=[]
    k=3
    for x in range(len(testSet)):
        neighbors = getNeighbours(trainingSet, testSet[x], k)
        result = getResponse(neighbors)
        predictions.append(result)
        print('> predicted='+repr(result)+', actual='+repr(testSet[x][-1]))
    accuracy = getAccuracy(testSet, predictions)
    print(accuracy)
    print('Accuracy: '+ repr(accuracy)+'%')
    
main() 

DataSet Size: 151
Train set: 103
Test set: 47
> predicted='Iris-setosa', actual='Iris-setosa'
> predicted='Iris-setosa', actual='Iris-setosa'
> predicted='Iris-setosa', actual='Iris-setosa'
> predicted='Iris-setosa', actual='Iris-setosa'
> predicted='Iris-setosa', actual='Iris-setosa'
> predicted='Iris-setosa', actual='Iris-setosa'
> predicted='Iris-setosa', actual='Iris-setosa'
> predicted='Iris-setosa', actual='Iris-setosa'
> predicted='Iris-setosa', actual='Iris-setosa'
> predicted='Iris-setosa', actual='Iris-setosa'
> predicted='Iris-setosa', actual='Iris-setosa'
> predicted='Iris-setosa', actual='Iris-setosa'
> predicted='Iris-setosa', actual='Iris-setosa'
> predicted='Iris-setosa', actual='Iris-setosa'
> predicted='Iris-setosa', actual='Iris-setosa'
> predicted='Iris-versicolor', actual='Iris-versicolor'
> predicted='Iris-versicolor', actual='Iris-versicolor'
> predicted='Iris-versicolor', actual='Iris-versicolor'
> predicted='Iris-versicolor', actual='Iris-versicolor'
> predicte